In [ ]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function 
## using for python 3.x keywords and features in python 2. > for compatibility 

from torch import nn, optim, cuda 
from torch.utils import data 
## in this module, there are Dataset and DataLoader ,, etc.
## data.Dataset >> is using for virtual class for custom data
## data.DataLoader >> is using for input Pipeline

from torchvision import datasets, transforms 
## The torchvision package consists of popular datasets, model architectures, 
## and common image transformations for computer vision.

import torch.nn.functional as F 
import time

#Training settings 
batch_size = 64 
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST on {device}\n {"="*40}')

#MNIST Dataset 
train_dataset = datasets.MNIST(root='./mnist_data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
test_dataset = datasets.MNIST(root='./mnist_data/',
                             train=False,
                             transform=transforms.ToTensor())

#Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                            batch_size = batch_size,
                            shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                             batch_size=batch_size,
                             shuffle=False)

class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        self.l1 = nn.Linear(784,520)
        self.l2 = nn.Linear(520,320)
        self.l3 = nn.Linear(320,240)
        self.l4 = nn.Linear(240,120)
        self.l5 = nn.Linear(120,10)
        
    def forward(self,x):
        x = x.view(-1,784) # Flatten the data (n,1,28,28) -> (n,784) 
        ## i think > n is batch_size,, (?)
        ## such as view of DataBase (?) 
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)
    
model = Net()
model.to(device)  
## i think > it send model to device
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.5)


def train(epoch):
    model.train()  # >> i think ,, overriding
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        #sending data and target to device
        
        '''
        Reference) 
        
        for i,data in enumerate(train_loader,0):
            #get the inputs 
            inputs, labels = data
        
        the data shape looks like,, 
        [tensor([[],[],...,[]]),tensor([[],[],[],...,[]])]
        for inputs and then for labels 
        
        >>>  list(tensor(inputs), tensor(data)) <<< 
        type(data) >> class 'list'
        type(data[0]) >> Torch.Tensor
        '''
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0: 
            print('Train Epoch : {} | Batch Status : {}/{} ({:.0f}%) | Loss : {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item())) 
            

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader : 
        data, target = data.to(device), target.to(device)
        output = model(data)
        #sum up batch loss 
        tset_loss += criterion(output,target).item()
        #get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print(f'======================\nTest set: Average loss : {test_loss:.4f},Accuracy:{correct}/{len(test_loader.dataset)}'
         f'({100.*correct / len(test_loader.dataset):.0f}%)')
    
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1,10):
        epoch_start = time.time()
        train(epoch)
        m,s = divmod(time.time() - epoch_start, 60)
        print(f'Training time : {m:.0f}m {s:.0f}s')
        
    m,s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s \n Model was trained on {device}!')
        
      


In [6]:
import torch
x = torch.randn(4,4)
y = x.view(16)
z = x.view(-1,8)

print(x,'\n',y,'\n',z)

tensor([[ 0.1663,  1.1857,  1.0400,  0.5672],
        [-1.5602, -0.8451,  0.3930, -0.1837],
        [ 0.2274, -0.4366, -1.2270, -0.2365],
        [-0.1978,  0.4887,  1.2223,  0.7826]]) 
 tensor([ 0.1663,  1.1857,  1.0400,  0.5672, -1.5602, -0.8451,  0.3930, -0.1837,
         0.2274, -0.4366, -1.2270, -0.2365, -0.1978,  0.4887,  1.2223,  0.7826]) 
 tensor([[ 0.1663,  1.1857,  1.0400,  0.5672, -1.5602, -0.8451,  0.3930, -0.1837],
        [ 0.2274, -0.4366, -1.2270, -0.2365, -0.1978,  0.4887,  1.2223,  0.7826]])
